In [1]:
import ollama

In [8]:
import ollama
from typing import List, Dict, Optional

from pydantic import BaseModel, Field
from typing import Literal, Dict, Any

class FunctionParameters(BaseModel):
    type: Literal["object"] = "object"
    properties: Dict[str, Dict[str, Any]]
    required: list[str]

class Function(BaseModel):
    name: str = Field(description="Name of the function to call")
    description: str = Field(description="Description of what the function does")
    parameters: FunctionParameters

class OllamaTool(BaseModel):
    type: Literal["function"] = "function"
    function: Function

# Example usage for search function
search_parameters = FunctionParameters(
    properties={
        "query": {
            "type": "string",
            "description": "The search query."
        }
    },
    required=["query"]
)

search_function = Function(
    name="get_search_results",
    description="Get search results for a provided query.",
    parameters=search_parameters
)

tools_schema = [OllamaTool(
    function=search_function
).model_dump()]

def ollama_function_selection(user_message: str,
                            tools_schema: List,
                            model_name: str = "llama3.1:8b") -> Optional[dict]:
    """
    Get function calling arguments from Ollama model for a single function call.
    Returns None if no function is selected.
    """
    messages=[{
        "role": "user", 
        "content": user_message
    }]
    
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )

    if not response["message"].get("tool_calls"):
        return None
    
    # Return arguments of first tool call
    tool = response["message"]["tool_calls"][0]
    return tool["function"]["arguments"]

In [9]:
ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

{'query': 'HNSW algorithm'}

In [10]:
ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.2:3b"
)

{'query': 'HNSW how it works'}

In [14]:
import ollama
from typing import List, Dict, Optional

from pydantic import BaseModel, Field
from typing import Literal, Dict, Any

class FunctionParameters(BaseModel):
    type: Literal["object"] = "object"
    properties: Dict[str, Dict[str, Any]]
    required: list[str]

class Function(BaseModel):
    name: str = Field(description="Name of the function to call")
    description: str = Field(description="Description of what the function does")
    parameters: FunctionParameters

class OllamaTool(BaseModel):
    type: Literal["function"] = "function"
    function: Function

class SearchQuery(BaseModel):
    query: str = Field(description="The search query to execute")
    target_collection: str = Field(description="The collection to search in", default="default")
    filters: str = Field(description="Any filters to apply to the search", default="")

# Get the schema for SearchQuery, which defines the query parameter structure
search_query_schema = SearchQuery.model_json_schema()
query_properties = search_query_schema["properties"]

# Example usage for search function
search_parameters = FunctionParameters(
    properties={
        "query": query_properties["query"],
        "target_collection": query_properties["target_collection"],
        "filters": query_properties["filters"]
    },
    required=["query", "target_collection"]  # Make both fields required
)

search_function = Function(
    name="get_search_results",
    description="Search for documents in a specified collection using a query and optional filters.",
    parameters=search_parameters
)

tools_schema = [OllamaTool(
    function=search_function
).model_dump()]

def ollama_function_selection(user_message: str,
                            tools_schema: List,
                            model_name: str = "llama3.1:8b") -> Optional[dict]:
    """
    Get function calling arguments from Ollama model for a single function call.
    Returns None if no function is selected.
    """
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant that searches through document collections. When given a query, you should specify both the search query and target collection."
    }, {
        "role": "user", 
        "content": user_message
    }]
    
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )

    if not response["message"].get("tool_calls"):
        return None
    
    # Return arguments of first tool call
    tool = response["message"]["tool_calls"][0]
    return tool["function"]["arguments"]

In [18]:
model_response = ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

print(model_response)

{'filters': '', 'query': 'HNSW algorithm', 'target_collection': 'Computer Science Documentation'}
